<a href="https://colab.research.google.com/github/mikebilly/hdm/blob/main/Video2x_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount and authenticate

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Initialize

In [ ]:
import glob
!apt install ffmpeg
!apt install libmagic1 python3-yaml
!apt install libvulkan-dev

!git clone https://github.com/k4yt3x/video2x.git
import os
os.chdir('video2x/src')

!git checkout master
!pip install -r requirements.txt
!rm -rf video2x.yaml
!wget -O video2x.yaml https://raw.githubusercontent.com/mikebilly/hdm/main/video2x.yaml

os.chdir('../..')
!wget https://github.com/nihui/waifu2x-ncnn-vulkan/releases/download/20210521/waifu2x-ncnn-vulkan-20210521-ubuntu.zip
!7z x waifu2x-ncnn-vulkan-20210521-ubuntu.zip
!pip install -U PyYAML
!rm /usr/local/lib/python3.7/dist-packages/PIL/Image.py
!wget -P /usr/local/lib/python3.7/dist-packages/PIL https://raw.githubusercontent.com/mikebilly/hdm/main/Image.py

os.mkdir('/content/input')
os.mkdir('/content/output')

import magic
func = magic.Magic() 

# Upload

In [ ]:
!wget -P /content/input 
scale = 2

# Upscale

In [ ]:
input_path = glob.glob("/content/input/*.*")[0]
mimestart = func.from_file(input_path).split(' ')[1]
if mimestart == "image":
  output_ext = ".png"
else:
  output_ext = ".mkv"

output_filename = os.path.splitext(os.path.basename(input_path))[0] + "_" + str(scale) + "x" + output_ext
output_path = "/content/output/" + output_filename
print(input_path)
print(output_path)
print(output_filename)

In [ ]:
os.chdir('/content/output')
!python /content/video2x/src/video2x.py -i $input_path -o $output_filename -d waifu2x_ncnn_vulkan -r $scale
!cp $output_path "/content/gdrive/MyDrive"


    __      __  _       _                  ___   __   __
    \ \    / / (_)     | |                |__ \  \ \ / /
     \ \  / /   _    __| |   ___    ___      ) |  \ V /
      \ \/ /   | |  / _` |  / _ \  / _ \    / /    > <
       \  /    | | | (_| | |  __/ | (_) |  / /_   / . \
        \/     |_|  \__,_|  \___|  \___/  |____| /_/ \_\


                  Video2X Video Enlarger

                       Version 4.3.3

[+] INFO: Loading files into processing queue
2021-12-10 07:59:15.094426 [+] INFO: Input path(s): /content/input/eye.png
2021-12-10 07:59:15.095192 [+] INFO: File MIME type: image/png
[+] INFO: Loaded files into processing queue
2021-12-10 07:59:15.096763 [+] INFO: Input file: /content/input/eye.png
[+] INFO: Reading file information
2021-12-10 07:59:15.096982 [+] INFO: Executing: /usr/bin/ffprobe -v quiet -print_format json -show_format -show_streams -i /content/input/eye.png
2021-12-10 07:59:15.390379 [+] INFO: Creating cache directory /tmp/video2x
2021-12-10 07:59:15.39

In [ ]:
!rm -rf /content/input
os.mkdir('/content/input')

# Download

In [ ]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in file_list:
  if file['title'] == output_filename:
    print("https://drive.google.com/file/d/{}".format(file['id']))
    break